# Laden der Daten
In diesem Notebook werden die Daten eingelesen, welche direkt über die URL zugängig sind. Die Geodaten für das Weg- und Velonetz <br> 
in Zürich müssen manuell auf der Open Data Seite runtergeladen und dann über einen CSV-Import geladen werden, nachdem die Datei Entzipped wurde.<br>
Grund hierfür ist die Auswahl des Ausschnitts, in welchem das Velonetz angezeigt werden soll. <br>
Dieser muss auf der Seite ausgewählt werden und es wird dann ein Downlaodlink für 5 Minuten zur Verfügung gestellt.


## Verwendete Links/Quellen

In [ ]:
url_x=
url_y=

# Installation notwendiger Bibliotheken

In [ ]:
!pip install --upgrade pip
!pip install folium
!pip install matplotlib
!pip install geopandas
!pip install geopy

# Module importieren

# Funktionen